# Working with Forms in Jupyter

##Inspired by:
###https://jakevdp.github.io/blog/2013/06/01/ipython-notebook-javascript-python-communication/

##Code modernized by reviewing slider code:<br/>
###http://nbviewer.ipython.org/github/jakevdp/mpld3/blob/gh-pages/notebooks/sliderPlugin.ipynb

#Changes
>Call to execute has not changed, but the information needed by the callback argument and information passed to the callback function has changed.  Additional information on messaging in iPython/Jupyter can be found [here](https://ipython.org/ipython-doc/stable/development/messaging.html).

    // 2.x:
    // Message was passed back in two arguments:
    //   out_type: type of output {stream, pyout, pyerr}
    //   out: returned output with additional metadata
    //      out_type='stream'  out.data was a stream of output
    //      out_type='pyout'   out.data["text/plain"] is the returned output
    //      out_type='pyerr'   out.ename/out.evalue contained error type and message
    function handle_output(out_type, out) {//handler code}
    var callbacks = {'output' : handle_output};
    
    // 3.x:
    // Everything is returned in one argument
    //   out.msg_type replaces out_type
    //     'error' replaces 'pyerr'
    //     'execute_result' replaces 'pyout'
    //     'stream' is 'stream'
    //   out.content now contains returned messages/output
    //     msg_type = 'error'
    //       out.content.ename = error name
    //       out.content.evalue = error value
    //       out.content.traceback = python traceback
    //     msg_type = 'execute_result'
    //       out.content.data['text/plain'] = returned output
    //     msg_type = 'stream' 
    //       if out.content.name = 'stdout' output is in out.content.text
    function handle_output(out) {//handler code}
    var callbacks = { 'iopub' : {'output' : handle_output}};
    
#Types
* 'execute_result' : A python object has been returned
* 'stream' : Output from a print statement
* 'error' : An error of some sort has occurred


In [1]:
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black; width:300px; padding:20px;">
Variable Name: <input type="text" id="var_name" value="foo"><br>
Variable Value: <input type="text" id="var_value" value="bar"><br>
<button onclick="set_value()">Set Value</button>
</div>
"""

javascript = """
<script type="text/Javascript">
    function set_value(){
        var var_name = document.getElementById('var_name').value;
        var var_value = document.getElementById('var_value').value;
        var command = var_name + " = '" + var_value + "'";
        console.log("Executing Command: " + command);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command);
    }
</script>
"""

HTML(input_form + javascript)


In [2]:
from math import pi, sin

In [2]:
# As a test put 'print(a)' into the Code: text box to test 'stream' return types
a=5
print(a)

5


In [3]:
# Add an input form similar to what we saw above
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black; width:600px; padding:20px;">
Code: <input type="text" id="code_input" size="50" height="2" value="sin(pi / 2)"><br>
Result: <input type="text" id="result_output" size="50" value="1.0"><br>
<button onclick="exec_code()">Execute</button>
</div>
"""

# here the javascript has a function to execute the code
# within the input box, and a callback to handle the output.
javascript = """
<script type="text/Javascript">
    function handle_output(out){
        console.log(out);
        var res = null;
        switch (out.msg_type) {
            case 'stream':
                if (out.content.name == "stdout") {
                    res = out.content.text;
                }
                break;
            case 'execute_result':
                res = out.content.data["text/plain"];
                break;
            case 'error':
                res = out.content.ename + ": " + out.content.evalue;
                break;
            default:
                res = '[Return type undefined: ' + out.msg_type + ' ]';
        }
        document.getElementById("result_output").value = res;
    }
    
    function exec_code(){
        var code_input = document.getElementById('code_input').value;
        var kernel = IPython.notebook.kernel;
        var callbacks = { 'iopub' : {'output' : handle_output}};
        document.getElementById("result_output").value = "";  // clear output box
        var msg_id = kernel.execute(code_input, callbacks, {silent:false});
        console.log("Execute pressed");
    }
</script>
"""

HTML(input_form + javascript)